In [184]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [185]:
def compute_depth(imageL, imageR, disp=128, block=15):
    stereo = cv2.StereoBM_create(numDisparities=disp, blockSize=block)

    grayL = cv2.blur(cv2.cvtColor(imageL, cv2.COLOR_BGR2GRAY), (5,5))
    grayR = cv2.blur(cv2.cvtColor(imageR, cv2.COLOR_BGR2GRAY), (5,5))

    disparity = stereo.compute(grayL, grayR)
    return disparity

In [186]:
def overlay_images(source, overlay, mask):
    mask = mask.astype(bool)
    if(len(mask.shape) == 2):
        mask = np.dstack((mask, mask, mask))

    result = np.copy(source)
    h, w = overlay.shape[:2]
    result[0:h, 0:w] = (overlay * mask) + (source[0:h, 0:w] * ~mask)

    return result

In [187]:
def load_configs(config_path):
    configs = {}
    with open(config_path, 'r') as f:
        lines = f.readlines()
        for line in lines:
            key, value = line.split(': ')
            configs[key.strip()] = np.array(value.strip().split(" ")).astype('float32').reshape(3,-1)
    return configs


In [188]:
def compute_projection_matrix(configs, shape):
    projection_matrix = np.zeros((4,4))
    cv2.stereoRectify(Q = projection_matrix,
                      cameraMatrix1 = configs["P2"][:,:3],
                      cameraMatrix2 = configs["P3"][:,:3],
                      imageSize = shape[:2],
                      R = np.identity(3),
                      T = np.array([0.54, 0., 0.]),
                      distCoeffs1 = 0, distCoeffs2 = 0, 
                      R1 = None, R2 = None, P1 =  None, P2 =  None)
    return projection_matrix

In [189]:
def compute_points_and_colors(points, left_image, disparity):
    reflect_matrix = np.identity(3)
    reflect_matrix[0] *= -1
    points = np.matmul(points,reflect_matrix)

    colors = cv2.cvtColor(left_image, cv2.COLOR_BGR2RGB)

    mask = disparity > disparity.min()
    verts = points[mask]
    colors = colors[mask]
    
    idx = np.fabs(verts[:,0]) < 4.5
    verts = verts[idx]
    colors = colors.reshape(-1, 3)
    colors = colors[idx]

    return verts, colors


In [190]:
def write_ply(fn, verts, colors):
    ply_header = '''ply
    format ascii 1.0
    element vertex %(vert_num)d
    property float x
    property float y
    property float z
    property uchar red
    property uchar green
    property uchar blue
    end_header
    '''
    out_colors = colors.copy()
    verts = verts.reshape(-1, 3)
    verts = np.hstack([verts, out_colors])
    with open(fn, 'wb') as f:
        f.write((ply_header % dict(vert_num=len(verts))).encode('utf-8'))
        np.savetxt(f, verts, fmt='%f %f %f %d %d %d ')

In [196]:
FOLDER = "Sample"

left = cv2.imread(f'images/{FOLDER}/L.png')
right = cv2.imread(f'images/{FOLDER}/R.png')
configs = load_configs(f'images/{FOLDER}/config.txt')

projection_matrix = compute_projection_matrix(configs, left.shape)

disparity = compute_depth(left, right, disp=16*7, block=11)

points = cv2.reprojectImageTo3D(disparity, projection_matrix)
verts, colors = compute_points_and_colors(points, left, disparity)

write_ply(f'results/plys/{FOLDER}.ply', verts, colors)


/tmp/ipykernel_5892/438800613.py:5: RuntimeWarning: invalid value encountered in matmul
  points = np.matmul(points,reflect_matrix)
